In [1]:
import numpy
import matplotlib.pyplot as plt

In [3]:
def update_weights_hebbian(w0):
    return w1

def update_weights_BCM(w0):
    return w1

def update_weights_hebbian_normalized(w0):
    return w1